In [3]:
import docker
import pandas as pd

def docker_to_dataframe():
    # Create a Docker client connected to the local Docker daemon
    client = docker.from_env()
    
    # Retrieve a list of images
    images = client.images.list()
    
    # Prepare data for DataFrame
    image_data = []
    for image in images:
        for tag in image.tags:
            repo, tag = tag.split(":") if ":" in tag else (tag, "latest")
            image_data.append({
                "REPOSITORY": repo,
                "TAG": tag,
                "IMAGE ID": image.id.split(":")[1][:12],
                "CREATED": image.attrs['Created'],
                "SIZE": image.attrs['Size']
            })
    
    # Create a DataFrame from the image data
    df1 = pd.DataFrame(image_data)
    
    containers = client.containers.list(all=True)
    
    # Prepare data for DataFrame
    container_data = []
    for container in containers:
        container_data.append({
            "CONTAINER ID": container.short_id,
            "IMAGE": container.image.tags[0] if container.image.tags else 'No tag',
            "COMMAND": container.attrs['Config']['Cmd'],
            "CREATED": container.attrs['Created'],
            "STATUS": container.status,
            "NAMES": container.name
        })
    
    # Create a DataFrame from the container data
    df2 = pd.DataFrame(container_data)
    
    return df1,df2

# Call the function and display the DataFrame
df1,df2 = docker_images_to_dataframe()

In [6]:
df1['IMAGE'] = df1['REPOSITORY']+":"+df1['TAG'] 

In [7]:
df1

,REPOSITORY,TAG,IMAGE ID,CREATED,SIZE,IMAGE
0,faris_test_v1,latest,8b887e727987,2024-09-18T10:25:19.179300017+03:00,823389415,faris_test_v1:latest
1,hello-world,latest,d2c94e258dcb,2023-05-02T16:49:27Z,13256,hello-world:latest


In [11]:
df2.merge(df1,on='IMAGE',how='left',suffixes=('_CONTAINER','_IMAGE'))

,CONTAINER ID,IMAGE,COMMAND,CREATED_CONTAINER,STATUS,NAMES,REPOSITORY,TAG,IMAGE ID,CREATED_IMAGE,SIZE
0,dae48ece0e90,hello-world:latest,[/hello],2024-09-18T11:46:21.250981946Z,exited,optimistic_wu,hello-world,latest,d2c94e258dcb,2023-05-02T16:49:27Z,13256
1,331b166feb1e,faris_test_v1:latest,"[uvicorn, main:app, --host, 0.0.0.0, --port, 8...",2024-09-18T07:25:29.215156412Z,exited,faris_test_v1,faris_test_v1,latest,8b887e727987,2024-09-18T10:25:19.179300017+03:00,823389415


In [2]:
import pandas as pd

data = pd.read_csv('test_data/child_data (5).csv')

In [5]:
data.sii

0       Moderate
1            NaN
2            NaN
3           Mild
4            NaN
          ...   
3955        Mild
3956         NaN
3957        Mild
3958         NaN
3959         NaN
Name: sii, Length: 3960, dtype: object

In [32]:
import requests
import json

def get_no_entity(record):
   url = "http://0.0.0.0:8000/no_of_entities_forecast_model/predict"

   payload = json.dumps({
   "data": [
      record
   ]
   })
   headers = {
   'Content-Type': 'application/json'
   }

   response = requests.request("POST", url, headers=headers, data=payload)
   print(response.text)
   return response.json()['predictions'][0]

def get_no_objects(record):
   url = "http://0.0.0.0:8001/no_of_objects_forecast_model/predict"

   payload = json.dumps({
   "data": [
      record
   ]
   })
   headers = {
   'Content-Type': 'application/json'
   }

   response = requests.request("POST", url, headers=headers, data=payload)
   print(response.text)
   return response.json()['predictions'][0]

def get_no_learners(record):
   url = "http://0.0.0.0:8002/no_of_learners_forecast_model/predict"

   payload = json.dumps({
   "data": [
      record
   ]
   })
   headers = {
   'Content-Type': 'application/json'
   }

   response = requests.request("POST", url, headers=headers, data=payload)
   print(response.text)
   return response.json()['predictions'][0]

In [44]:
import pandas as pd
data = pd.read_csv('test_data/to_forecast.csv')

In [45]:
data = data.to_dict('records')

In [46]:
for i in range(6):
    last_date_record = data[-1].copy()
    print(last_date_record)
    del last_date_record['year']
    del last_date_record['No_Of_Entities']
    del last_date_record['No_Of_Objects']
    del last_date_record['No_Of_Learners']
    last_date_record = {}
    last_date_record['month'] = data[-1]['month']+1
    if last_date_record['month'] > 12:
        last_date_record['month'] = last_date_record['month'] - 12
    last_date_record['month_lag_1'] = data[-1]['month']
    last_date_record['month_lag_2'] = data[-1]['month_lag_1']
    last_date_record['month_lag_3'] = data[-1]['month_lag_2']
    last_date_record['month_lag_4'] = data[-1]['month_lag_3']
    last_date_record['No_Of_Entities_lag_1'] = data[-1]['No_Of_Entities']
    last_date_record['No_Of_Objects_lag_1'] = data[-1]['No_Of_Objects']
    last_date_record['No_Of_Learners_lag_1'] = data[-1]['No_Of_Learners']
    last_date_record['No_Of_Entities_lag_2'] = data[-1]['No_Of_Entities_lag_1']
    last_date_record['No_Of_Objects_lag_2'] = data[-1]['No_Of_Objects_lag_1']
    last_date_record['No_Of_Learners_lag_2'] = data[-1]['No_Of_Learners_lag_1']
    last_date_record['No_Of_Entities_lag_3'] = data[-1]['No_Of_Entities_lag_2']
    last_date_record['No_Of_Objects_lag_3'] = data[-1]['No_Of_Objects_lag_2']
    last_date_record['No_Of_Learners_lag_3'] = data[-1]['No_Of_Learners_lag_2']
    last_date_record['No_Of_Entities_lag_4'] = data[-1]['No_Of_Entities_lag_3']
    last_date_record['No_Of_Objects_lag_4'] = data[-1]['No_Of_Objects_lag_3']
    last_date_record['No_Of_Learners_lag_4'] = data[-1]['No_Of_Learners_lag_3']
    en = get_no_entity(last_date_record)
    obj = get_no_objects(last_date_record)
    lrs =get_no_learners(last_date_record)
    last_date_record['No_Of_Entities'] = en
    last_date_record['No_Of_Objects'] = obj
    last_date_record['No_Of_Learners'] = lrs
    last_date_record['year'] = 2025
    data.append(last_date_record)

{'year': 2024, 'month': 10, 'No_Of_Entities': 160.0, 'No_Of_Objects': 2862.0, 'No_Of_Learners': 40151.0, 'date': '2024-10-01', 'month_lag_1': 9, 'No_Of_Entities_lag_1': 166.0, 'No_Of_Objects_lag_1': 3439.0, 'No_Of_Learners_lag_1': 47209.0, 'month_lag_2': 8, 'No_Of_Entities_lag_2': 226.0, 'No_Of_Objects_lag_2': 3265.0, 'No_Of_Learners_lag_2': 43249.0, 'month_lag_3': 7, 'No_Of_Entities_lag_3': 153.0, 'No_Of_Objects_lag_3': 2092.0, 'No_Of_Learners_lag_3': 20024.0, 'month_lag_4': 6, 'No_Of_Entities_lag_4': 132.0, 'No_Of_Objects_lag_4': 1912.0, 'No_Of_Learners_lag_4': 25885.0}
{"predictions":[220.78385198196236]}
{"predictions":[2680.4285714285716]}
{"predictions":[42578.2]}
{'month': 11, 'month_lag_1': 10, 'month_lag_2': 9, 'month_lag_3': 8, 'month_lag_4': 7, 'No_Of_Entities_lag_1': 160.0, 'No_Of_Objects_lag_1': 2862.0, 'No_Of_Learners_lag_1': 40151.0, 'No_Of_Entities_lag_2': 166.0, 'No_Of_Objects_lag_2': 3439.0, 'No_Of_Learners_lag_2': 47209.0, 'No_Of_Entities_lag_3': 226.0, 'No_Of_Object

In [49]:
pd.DataFrame(data)[['year','month','No_Of_Entities','No_Of_Objects','No_Of_Learners']].to_csv('/home/faris1/Desktop/forecasted_lrs_data.csv',index=False)

In [50]:
pd.DataFrame(data)

,year,month,No_Of_Entities,No_Of_Objects,No_Of_Learners,date,month_lag_1,No_Of_Entities_lag_1,No_Of_Objects_lag_1,No_Of_Learners_lag_1,...,No_Of_Objects_lag_2,No_Of_Learners_lag_2,month_lag_3,No_Of_Entities_lag_3,No_Of_Objects_lag_3,No_Of_Learners_lag_3,month_lag_4,No_Of_Entities_lag_4,No_Of_Objects_lag_4,No_Of_Learners_lag_4
0,2022,12,30.000000,298.000000,1764.0,2022-12-01,11,29.000000,325.000000,2567.0,...,590.000000,9202.0,9,21.000000,339.000000,2062.0,8,4.000000,4.000000,6.0
1,2023,1,47.000000,2739.000000,17705.0,2023-01-01,12,30.000000,298.000000,1764.0,...,325.000000,2567.0,10,34.000000,590.000000,9202.0,9,21.000000,339.000000,2062.0
2,2023,2,39.000000,4663.000000,12598.0,2023-02-01,1,47.000000,2739.000000,17705.0,...,298.000000,1764.0,11,29.000000,325.000000,2567.0,10,34.000000,590.000000,9202.0
3,2023,3,49.000000,3641.000000,17576.0,2023-03-01,2,39.000000,4663.000000,12598.0,...,2739.000000,17705.0,12,30.000000,298.000000,1764.0,11,29.000000,325.000000,2567.0
4,2023,4,47.000000,2189.000000,9224.0,2023-04-01,3,49.000000,3641.000000,17576.0,...,4663.000000,12598.0,1,47.000000,2739.000000,17705.0,12,30.000000,298.000000,1764.0
5,2023,5,65.000000,2061.000000,12100.0,2023-05-01,4,47.000000,2189.000000,9224.0,...,3641.000000,17576.0,2,39.000000,4663.000000,12598.0,1,47.000000,2739.000000,17705.0
6,2023,6,65.000000,1534.000000,10529.0,2023-06-01,5,65.000000,2061.000000,12100.0,...,2189.000000,9224.0,3,49.000000,3641.000000,17576.0,2,39.000000,4663.000000,12598.0
7,2023,7,87.000000,1044.000000,8286.0,2023-07-01,6,65.000000,1534.000000,10529.0,...,2061.000000,12100.0,4,47.000000,2189.000000,9224.0,3,49.000000,3641.000000,17576.0
8,2023,8,84.000000,1553.000000,8643.0,2023-08-01,7,87.000000,1044.000000,8286.0,...,1534.000000,10529.0,5,65.000000,2061.000000,12100.0,4,47.000000,2189.000000,9224.0
9,2023,9,77.000000,1901.000000,12504.0,2023-09-01,8,84.000000,1553.000000,8643.0,...,1044.000000,8286.0,6,65.000000,1534.000000,10529.0,5,65.000000,2061.000000,12100.0


In [52]:
"(hi)".index(")")

3